<a href="https://colab.research.google.com/github/vz415/AE32D_optimization/blob/master/MNIST_Generalized_AE_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# Getting classy
# import the necessary packages
# from tensorflow.keras.layers import LeakyReLU
# from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from keras import regularizers
# from tensorflow.keras import backend as K
import numpy as np

class flatAutoencoder:
  @staticmethod
  def build(input_shape, layer_dims=(512, 256, 128, 32), 
            enc_dropout_reg = None, dec_dropout_reg = None,
            enc_l2_reg = None, dec_l2_reg = None,
            enc_l1_reg = None, dec_l1_reg = None):
    '''
    enc_dropout_reg: takes tuple size of layer_dims and with the desired dropout
    in the correct index following encoding layer. e.g. (0.5,None,None) if a 
    dropout of 0.5 is desired after the initial layer in given default layer_dims.
    dec_dropout_reg: takes tuple size of layer_dims and desired dropout in 
    index following decoder layers. e.g.(0.5,None,None) in given default
    layer_dims. Currently can't do the last layer but that's small as most models
    performed worse with that layer.

    enc_l2_reg: l2 regularization added to encoder layers if specified. Takes a 
    tuple e.g. (10e-5, None, None) for regularization of the corresponding layer.
    dec_l2_reg: l2 regularization added to decoder layers if specified. Takes a 
    tuple e.g. (10e-5, None, None) for regularization of the corresponding layer.

    enc_l1_reg: l1 regularization added to encoder layers if specified. Takes a 
    tuple e.g. (10e-5, None, None) for regularization of the corresponding layer.
    dec_l1_reg: l1 regularization added to decoder layers if specified. Takes a 
    tuple e.g. (10e-5, None, None) for regularization of the corresponding layer.
    '''
    # define the input to the encoder
    inputs = Input(shape=(input_shape,))
    x = inputs

		# loop over the number of layer_dims to add sets of layers
    for i, dim in enumerate(layer_dims):
      # Check l2 regularization
      if enc_l2_reg is not None:
        if enc_l2_reg[i]:
          x = Dense(dim, activation='relu', 
                    activity_regularizer=regularizers.l2(enc_l2_reg[i]))(x)
        else:
          x = Dense(dim, activation='relu', 
                      activity_regularizer=None)(x)
      
      # Check l1 regularization
      elif enc_l1_reg is not None:
        if enc_l1_reg[i]:
          x = Dense(dim, activation='relu', 
                    activity_regularizer=regularizers.l1(enc_l1_reg[i]))(x)
        else:
          x = Dense(dim, activation='relu', 
                      activity_regularizer=None)(x)

      # Non-regularized layer if no l2/l1
      else:
        x = Dense(dim, activation='relu', 
                      activity_regularizer=None)(x)

      # Introduce dropout if specified
      if enc_dropout_reg is not None:
        if enc_dropout_reg[i]:
          x = Dropout(enc_dropout_reg[i])(x)
    
    # build encoder model
    encoder = Model(inputs, x, name="encoder")
    
    # Building decoder model
    latentInputs = Input(shape=(encodingDim,))
    x = latentInputs

    # loop over our number of layer_dims again, but in reverse order
    for i, dim in enumerate(layer_dims[::-1]):
      # Introduce dropout if specified
      if dec_dropout_reg is not None:
        if dec_dropout_reg[i]:
          x = Dropout(dec_dropout_reg[i])(x)

      # Check l2 regularization
      if dec_l2_reg is not None:
        if dec_l2_reg[i]:
          x = Dense(dim, activation='relu', 
                    activity_regularizer=regularizers.l2(dec_l2_reg[i]))(x)
        else:
          x = Dense(dim, activation='relu', 
                      activity_regularizer=None)(x)
      
      # Check l1 regularization
      elif dec_l1_reg is not None:
        if dec_l1_reg[i]:
          x = Dense(dim, activation='relu', 
                    activity_regularizer=regularizers.l1(dec_l1_reg[i]))(x)
        else:
          x = Dense(dim, activation='relu', 
                      activity_regularizer=None)(x)

      # Non-regularized layer if no l2/l1
      else:
        x = Dense(dim, activation='relu', 
                      activity_regularizer=None)(x)
      # x = LeakyReLU(alpha=0.2)(x) -> might want to try this out later
      
    # apply linear activation (in this case) for output
    outputs = Dense(input_shape, activation='linear')(x)
    
    # build the decoder model
    decoder = Model(latentInputs, outputs, name="decoder")

    # autoencoder is the encoder wrapped by decoder
    autoencoder = Model(inputs, decoder(encoder(inputs)),
                        name="autoencoder")

    # return a 3-tuple of the encoder, decoder, and autoencoder
    return (encoder, decoder, autoencoder)

TODO:
1. ~Create method to integrate regularization into encoding or decoding layer~
2. ~Fix error that makes layers smaller for some reason (see below layer summaries)~
3. Change layer_dims so that it includes the encoding dimension to simplify the code. Ridiculous that I would need to make a special argument just for the encoding dimension - it's the smallest layer. Just use that info.

Verify that regularization is built into the layer
4. Create optional argument to share parameters betwee encoding/decoding layers
5. 

Future work:
1. Make tests for if more dropouts are provided than can be used
2. Create error if regularization are put in the same spot in the network - e.g. whenever the tupes for both l1 and l2 encoder regularzation are the same
3. Allow different outputs from the model, such as softmax
4. Implement elastic net regularization

In [25]:
(encoder, decoder, autoencoder) = flatAutoencoder.build(1728, 
                                      enc_dropout_reg=(0.5,0.1,0.1,None), 
                                      dec_dropout_reg=(0.5,0.1,0.1,None),
                                      enc_l2_reg=(10e-5, 3, 6, None),
                                      dec_l1_reg=(10e-5, 4, 5, None)) #(10e-5, None, None)
print(encoder.summary())
print(decoder.summary())
print(autoencoder.summary())

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 1728)]            0         
_________________________________________________________________
dense_48 (Dense)             (None, 512)               885248    
_________________________________________________________________
dropout_31 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_49 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_32 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_50 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_33 (Dropout)         (None, 128)               0   

In [26]:
for layer in encoder.layers:
  # print(layer)
  print(layer.get_config())

{'batch_input_shape': (None, 1728), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_14'}
{'name': 'dense_48', 'trainable': True, 'dtype': 'float32', 'units': 512, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': {'class_name': 'L1L2', 'config': {'l1': 0.0, 'l2': 9.999999747378752e-05}}, 'kernel_constraint': None, 'bias_constraint': None}
{'name': 'dropout_31', 'trainable': True, 'dtype': 'float32', 'rate': 0.5, 'noise_shape': None, 'seed': None}
{'name': 'dense_49', 'trainable': True, 'dtype': 'float32', 'units': 256, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_re

In [18]:
for layer in decoder.layers:
  # print(layer)
  print(layer.get_config())

{'batch_input_shape': (None, 32), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_6'}
{'name': 'dropout_7', 'trainable': True, 'dtype': 'float32', 'rate': 0.5, 'noise_shape': None, 'seed': None}
{'name': 'dense_18', 'trainable': True, 'dtype': 'float32', 'units': 128, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': {'class_name': 'L1L2', 'config': {'l1': 9.999999747378752e-05, 'l2': 0.0}}, 'kernel_constraint': None, 'bias_constraint': None}
{'name': 'dropout_8', 'trainable': True, 'dtype': 'float32', 'rate': 0.1, 'noise_shape': None, 'seed': None}
{'name': 'dense_19', 'trainable': True, 'dtype': 'float32', 'units': 256, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': 

I really should look into making a CNN for this. So many less parameters with the CNN.

I now need to:
1. figure out how to choose where and what type of regularization occurs for each layer. The logic of the for loop is going to get a little more complicated by having 

In [ ]:
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import Dropout

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(784,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu',
                activity_regularizer=None)(input_img)
encoded = Dropout(0.4)(encoded)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(784, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

autoencoder.summary()

# autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# autoencoder.fit(x_train, x_train,
#                 epochs=50,
#                 batch_size=256,
#                 shuffle=True,
#                 validation_data=(x_test, x_test))

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        (None, 784)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 32)                25120     
_________________________________________________________________
dropout_14 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_44 (Dense)             (None, 784)               25872     
Total params: 50,992
Trainable params: 50,992
Non-trainable params: 0
_________________________________________________________________
